In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
model = InceptionV3(weights='imagenet')

96116736/96112376 [==============================] - 354s 4us/step


In [6]:
model_transform =  Model(model.input, model.layers[-2].output)

In [7]:
model_transform.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [13]:
encode('test.jpg').shape

(2048,)

In [11]:
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_transform.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [102]:
model = Sequential()
model.add(Dense(32, input_dim=2048))
model.add(Activation('tanh'))
model.add(Dense(10, input_dim=2048))
model.add(Activation('softmax'))

In [94]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                65568     
_________________________________________________________________
activation_99 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_100 (Activation)  (None, 10)                0         
Total params: 65,898
Trainable params: 65,898
Non-trainable params: 0
_________________________________________________________________


In [95]:
def get_path(key):
    len1 = len(str(key))
    str2 = "0"*(12-len1) + str(key)
    return str2

In [84]:
get_path(532481)

'000000532481'

In [50]:
def prepare_training_data(lda_result_file , directory_name ):
    f = open(lda_result_file,"rb")
    dict_lda = load(f)
    train_x = []
    train_y = []
    #print(dict_lda)
    for index,key in enumerate(dict_lda.keys()):
        #print("key",key,"index",index)
        image_path = directory_name + "/" + get_path(key)+".jpg"
        image_vector = encode(image_path)
        train_x.append( list(image_vector))
        train_y.append(list(dict_lda[key]))
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    
    return train_x,train_y

In [51]:
train_x , train_y = prepare_training_data("ldaResult","val2017")

In [52]:
print("YESSSSSSSSSSS")

YESSSSSSSSSSS


In [58]:
# with open("train_x","wb") as f:
#     dump(train_x,f)

In [59]:
# with open("train_y","wb") as f:
#     dump(train_y,f)

In [105]:
adam_opt = optimizers.adam(lr=0.001)
model.compile(optimizer=adam_opt , loss="categorical_crossentropy",metrics=['accuracy'])

In [106]:
model.fit(train_x,train_y,epochs=1000,verbose=True)

Epoch 1/1000
5000/5000 [==============================] - 2s 371us/step - loss: 0.6910 - acc: 0.9040
Epoch 2/1000
5000/5000 [==============================] - 0s 67us/step - loss: 0.6920 - acc: 0.8970
Epoch 3/1000
5000/5000 [==============================] - 0s 69us/step - loss: 0.6909 - acc: 0.9000
Epoch 4/1000
5000/5000 [==============================] - 0s 67us/step - loss: 0.6812 - acc: 0.9096
Epoch 5/1000
5000/5000 [==============================] - 0s 68us/step - loss: 0.6849 - acc: 0.9022
Epoch 6/1000
5000/5000 [==============================] - 0s 67us/step - loss: 0.6839 - acc: 0.8984
Epoch 7/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6806 - acc: 0.9070
Epoch 8/1000
5000/5000 [==============================] - 0s 70us/step - loss: 0.6907 - acc: 0.8928
Epoch 9/1000
5000/5000 [==============================] - 0s 70us/step - loss: 0.6835 - acc: 0.9014
Epoch 10/1000
5000/5000 [==============================] - 0s 68us/step - loss: 0.6790 - acc: 0.901

5000/5000 [==============================] - 0s 68us/step - loss: 0.6363 - acc: 0.9000
Epoch 83/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6335 - acc: 0.9002
Epoch 84/1000
5000/5000 [==============================] - 0s 73us/step - loss: 0.6309 - acc: 0.9096
Epoch 85/1000
5000/5000 [==============================] - 0s 78us/step - loss: 0.6411 - acc: 0.9036
Epoch 86/1000
5000/5000 [==============================] - 0s 73us/step - loss: 0.6273 - acc: 0.9192
Epoch 87/1000
5000/5000 [==============================] - 0s 76us/step - loss: 0.6298 - acc: 0.9158
Epoch 88/1000
5000/5000 [==============================] - 0s 77us/step - loss: 0.6285 - acc: 0.9110
Epoch 89/1000
5000/5000 [==============================] - 0s 69us/step - loss: 0.6262 - acc: 0.9122
Epoch 90/1000
5000/5000 [==============================] - 0s 68us/step - loss: 0.6308 - acc: 0.9112
Epoch 91/1000
5000/5000 [==============================] - 0s 73us/step - loss: 0.6278 - acc: 0.9142
Epoc

5000/5000 [==============================] - 0s 64us/step - loss: 0.6116 - acc: 0.9256
Epoch 163/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6146 - acc: 0.9164
Epoch 164/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6245 - acc: 0.9018
Epoch 165/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6203 - acc: 0.9116
Epoch 166/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6208 - acc: 0.9050
Epoch 167/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6131 - acc: 0.9154
Epoch 168/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6097 - acc: 0.9288
Epoch 169/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6102 - acc: 0.9276
Epoch 170/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6110 - acc: 0.9272
Epoch 171/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6126 - acc: 0.

5000/5000 [==============================] - 0s 64us/step - loss: 0.6055 - acc: 0.9312
Epoch 243/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.6031 - acc: 0.9344
Epoch 244/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6052 - acc: 0.9266
Epoch 245/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6039 - acc: 0.9286
Epoch 246/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.6053 - acc: 0.9302
Epoch 247/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.6065 - acc: 0.9264
Epoch 248/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6051 - acc: 0.9226
Epoch 249/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6036 - acc: 0.9258
Epoch 250/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6031 - acc: 0.9338
Epoch 251/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6025 - acc: 0.

5000/5000 [==============================] - 0s 65us/step - loss: 0.5990 - acc: 0.9354
Epoch 323/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.6059 - acc: 0.9258
Epoch 324/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.6054 - acc: 0.9176
Epoch 325/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6044 - acc: 0.9256
Epoch 326/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.6052 - acc: 0.9266
Epoch 327/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6009 - acc: 0.9326
Epoch 328/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5998 - acc: 0.9346
Epoch 329/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.5981 - acc: 0.9310
Epoch 330/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5973 - acc: 0.9406
Epoch 331/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.6003 - acc: 0.

...

5000/5000 [==============================] - 0s 73us/step - loss: 0.5942 - acc: 0.9408
Epoch 643/1000
5000/5000 [==============================] - 0s 70us/step - loss: 0.5958 - acc: 0.9352
Epoch 644/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5950 - acc: 0.9440
Epoch 645/1000
5000/5000 [==============================] - 0s 66us/step - loss: 0.5971 - acc: 0.9374
Epoch 646/1000
5000/5000 [==============================] - 0s 66us/step - loss: 0.5992 - acc: 0.9336
Epoch 647/1000
5000/5000 [==============================] - 0s 79us/step - loss: 0.5973 - acc: 0.9346
Epoch 648/1000
5000/5000 [==============================] - 0s 71us/step - loss: 0.5958 - acc: 0.9332
Epoch 649/1000
5000/5000 [==============================] - 0s 74us/step - loss: 0.5962 - acc: 0.9394
Epoch 650/1000
5000/5000 [==============================] - 0s 76us/step - loss: 0.5959 - acc: 0.9410
Epoch 651/1000
5000/5000 [==============================] - 0s 74us/step - loss: 0.5962 - acc: 0.

5000/5000 [==============================] - 0s 70us/step - loss: 0.5949 - acc: 0.9398
Epoch 723/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5942 - acc: 0.9440
Epoch 724/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5946 - acc: 0.9370
Epoch 725/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5945 - acc: 0.9392
Epoch 726/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5960 - acc: 0.9370
Epoch 727/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.5948 - acc: 0.9374
Epoch 728/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5988 - acc: 0.9274
Epoch 729/1000
5000/5000 [==============================] - 0s 66us/step - loss: 0.5976 - acc: 0.9390
Epoch 730/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5978 - acc: 0.9358
Epoch 731/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5948 - acc: 0.

5000/5000 [==============================] - 0s 65us/step - loss: 0.5942 - acc: 0.9392
Epoch 803/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5947 - acc: 0.9392
Epoch 804/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5941 - acc: 0.9392
Epoch 805/1000
5000/5000 [==============================] - 0s 67us/step - loss: 0.5952 - acc: 0.9372
Epoch 806/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.6001 - acc: 0.9290
Epoch 807/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5988 - acc: 0.9330
Epoch 808/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5931 - acc: 0.9418
Epoch 809/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.5937 - acc: 0.9396
Epoch 810/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5939 - acc: 0.9380
Epoch 811/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5937 - acc: 0.

5000/5000 [==============================] - 0s 72us/step - loss: 0.5941 - acc: 0.9454
Epoch 883/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5949 - acc: 0.9446
Epoch 884/1000
5000/5000 [==============================] - 0s 66us/step - loss: 0.5957 - acc: 0.9336
Epoch 885/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5956 - acc: 0.9346
Epoch 886/1000
5000/5000 [==============================] - 0s 67us/step - loss: 0.5963 - acc: 0.9346
Epoch 887/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5956 - acc: 0.9360
Epoch 888/1000
5000/5000 [==============================] - 0s 75us/step - loss: 0.5942 - acc: 0.9400
Epoch 889/1000
5000/5000 [==============================] - 0s 68us/step - loss: 0.5936 - acc: 0.9450
Epoch 890/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5941 - acc: 0.9394
Epoch 891/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.5937 - acc: 0.

5000/5000 [==============================] - 0s 66us/step - loss: 0.5944 - acc: 0.9448
Epoch 963/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5958 - acc: 0.9344
Epoch 964/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.5965 - acc: 0.9374
Epoch 965/1000
5000/5000 [==============================] - 0s 62us/step - loss: 0.5963 - acc: 0.9372
Epoch 966/1000
5000/5000 [==============================] - 0s 67us/step - loss: 0.5951 - acc: 0.9396
Epoch 967/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5934 - acc: 0.9400
Epoch 968/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5940 - acc: 0.9364
Epoch 969/1000
5000/5000 [==============================] - 0s 64us/step - loss: 0.5945 - acc: 0.9412
Epoch 970/1000
5000/5000 [==============================] - 0s 63us/step - loss: 0.5946 - acc: 0.9358
Epoch 971/1000
5000/5000 [==============================] - 0s 65us/step - loss: 0.5930 - acc: 0.

In [127]:
def predict_test_image(image_path):
    image_vector = encode(image_path)
    image_vector_1 = np.array([image_vector])
    print(image_vector_1.shape)
    probas = model.predict(image_vector_1)
    return probas

In [135]:
predict_test_image("tyagi.jpg")[0]

(1, 2048)


array([2.1891487e-05, 8.7331643e-04, 1.7074140e-04, 5.4652702e-02,
       8.6284279e-05, 2.1374390e-04, 1.8307652e-05, 9.4345260e-01,
       3.9821817e-04, 1.1218027e-04], dtype=float32)

In [109]:
f = open('topic_word_prob','rb')
g = load(f)
g[0]

{'bowl': 0.02197122,
 'eat': 0.026833424,
 'food': 0.03123857,
 'glass': 0.031358887,
 'group': 0.038096927,
 'orange': 0.011274825,
 'people': 0.052405376,
 'sit': 0.046688832,
 'table': 0.09352577,
 'wine': 0.019313678}

In [138]:
def predict_words_probablity(image_path , dictionary_file_name , topic_word_file,  num_of_topics ):
    f = open(topic_word_file , 'rb')
    topic_word_prob = load(f)
    all_unique_words_in_topics = []
    final_probablity = {}
    
    probality_topic_image = list(predict_test_image(image_path))
    
    for topic_num in topic_word_prob.keys():
        for word in topic_word_prob[topic_num].keys():
            if word not in all_unique_words_in_topics:
                all_unique_words_in_topics.append(word)
                
    for word in all_unique_words_in_topics:
        
        for topic in topic_word_prob.keys():
            
            #print(topic,word)
            if word not in final_probablity:
                final_probablity[word] = 0
            
            if word in topic_word_prob[topic]:
                final_probablity[word] += topic_word_prob[topic][word]*probality_topic_image[0][topic]
    
    sorted_probs_word_text = [(k, final_probablity[k]) for k in sorted(final_probablity, key=final_probablity.get, reverse=True)]
    return sorted_probs_word_text

In [144]:
predict_words_probablity("83085.jpg",'Dict','topic_word_prob',10)

(1, 2048)


[('people', 0.0631325771671527),
 ('fly', 0.04046446830034256),
 ('train', 0.03931855410337448),
 ('kite', 0.036848388612270355),
 ('beach', 0.033674780279397964),
 ('water', 0.030073486268520355),
 ('stand', 0.02807271176251369),
 ('group', 0.017737267633719966),
 ('walk', 0.016967670992016792),
 ('kit', 0.01684562861919403),
 ('clock', 0.0026222290471196175),
 ('tower', 0.0013110693544149399),
 ('field', 0.0011596783297136426),
 ('build', 0.0009805466197576607),
 ('grass', 0.0009250514558516443),
 ('large', 0.000921573315281421),
 ('tree', 0.0006474492838606238),
 ('fence', 0.0006206401158124208),
 ('giraffe', 0.0005468108574859798),
 ('tennis', 0.0002014602068811655),
 ('play', 0.00016040768241509795),
 ('baseball', 0.00014882029790896922),
 ('game', 0.00012661388609558344),
 ('ball', 0.0001018410621327348),
 ('hold', 9.354104315661971e-05),
 ('player', 8.464027632726356e-05),
 ('court', 6.598611798835918e-05),
 ('swing', 5.593043169938028e-05),
 ('sit', 4.524348040035875e-05),
 ('s